In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.extend([
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject',
    'C:/Users/Haakon/OneDrive/Dokumenter/FORSKNING/mastersproject/src/mastersproject/GTS'
])

# Notebook images
from IPython.display import Image

In [2]:
import GTS as gts
# from GTS.ISC_data.isc import ISCData
import numpy as np
import pandas as pd

In [3]:
cls = gts.ISCData()

In [15]:
# Import shear-zone -- borehole data.
sz_bh = cls.shearzone_borehole_data()
sz_bh = sz_bh[sz_bh['depth'].notna()]  # Remove rows with no intersection.
sz_bh.head(3)

,borehole,depth,shearzone
0,INJ1,34.92,S1_1
1,INJ2,30.99,S1_1
2,FBS1,31.09,S1_1


In [19]:
strc = cls.borehole_structure_data().merge(cls.borehole_data(), how='outer', on='borehole', 
                                           suffixes=('_struc', '_bh'), validate='m:1')
# strc['shearzone'] = np.nan
strc1 = strc.loc[:,['depth', 'type', 'borehole']]
strc1.head()

,depth,type,borehole
0,2.35,Minor ductile Shear-zone,FBS1
1,2.99,Minor ductile Shear-zone,FBS1
2,3.27,Foliation,FBS1
3,5.02,Minor ductile Shear-zone,FBS1
4,6.76,Foliation,FBS1


In [25]:
strc1.shape

(591, 3)

In [22]:
full_df = pd.merge_asof(strc.sort_values('depth'),
                  sz_bh.sort_values('depth'),
                  by='borehole', 
                  on='depth',
                  tolerance=0.01, 
                  direction='nearest')
df_no_na = full_df[full_df.shearzone.notna()]
# t3 = tt[tt['borehole'] == 'PRP3']

df_filtered = df_no_na[df_no_na.type.isin(['S1 Shear-zone', 'S3 Shear-zone', 'Minor ductile Shear-zone'])]
df_filtered[df_filtered['borehole'] == 'PRP3']

,depth,azimuth_struc,dip,aperture,type,borehole,x,y,z,length,diameter,azimuth_bh,upward_gradient,shearzone
151,15.57,185.46,60.58,308.85,S3 Shear-zone,PRP3,667468.39,158892.66,1733.1,32.33,131,335.01,-25.0,S3_1
216,19.34,166.63,56.55,315.75,S3 Shear-zone,PRP3,667468.39,158892.66,1733.1,32.33,131,335.01,-25.0,S3_2
336,25.57,176.50,55.72,1539.04,S1 Shear-zone,PRP3,667468.39,158892.66,1733.1,32.33,131,335.01,-25.0,S1_1
415,31.14,166.40,54.90,1201.46,S1 Shear-zone,PRP3,667468.39,158892.66,1733.1,32.33,131,335.01,-25.0,S1_2


In [24]:
full_df.shape

(591, 14)

In [27]:
df_filtered.shape

(39, 14)

In [12]:
# The mask can be used to save the index
_mask_sz = strc.loc[:,'type'].isin(['Minor ductile Shear-zone', 
                                    'S1 Shear-zone', 
                                    'S3 Shear-zone']).values

# Original index we have reduced to
sz_idx = strc1.index[_mask_sz]

# Reduced problem: only shear-zone like rows.
sz1 = strc1[_mask_sz]